In [1]:
import arcpy
arcpy.env.workspace = r'E:\My Drive\School\College\06 Grad Full\Semester 2\Geospatial Programming\Assignment 3/hw3.gdb'
arcpy.env.overwriteOutput = True

In [2]:
fcList = arcpy.ListFeatureClasses()
for fc in fcList:
    print(fc)

bike_routes
facilities
hospitals
parks
zip_areas
block_groups
thiessen_hosp
facilities_in_block_group


In [3]:
block_groups = "block_groups"
facilities = 'facilities'
outfc = 'facilities_in_block_group'

# arcpy.SpatialJoin_analysis(block_groups, facilities, outfc, '#', '#', '#', 'CONTAINS')

In [4]:
hospitalFc = 'hospitals'
thiessen_hospital = 'thiessen_hosp'
# arcpy.CreateThiessenPolygons_analysis(hospitalFc, thiessen_hospital, 'ALL')

In [5]:
# switch to assignment 6 dataset for IDW and kriging
folder = r'E:\My Drive\School\College\06 Grad Full\Semester 2\Geospatial Programming\Assignment 6'
arcpy.env.workspace = folder

In [12]:
arcpy.CreateFileGDB_management(arcpy.env.workspace, 'interpolation.gdb')

<Result 'E:\\My Drive\\School\\College\\06 Grad Full\\Semester 2\\Geospatial Programming\\Assignment 6\\interpolation.gdb'>

In [6]:
arcpy.env.workspace = 'interpolation.gdb'

In [15]:
## convert table with yearly numbers to ArcGIS friendly table (except IA0000, that's the state overall)
inTable = folder + "/yearly.csv"
outTable = 'yearly'
expression = arcpy.AddFieldDelimiters(arcpy.env.workspace, "stationID") + " <> 'IA0000'"

arcpy.TableToTable_conversion(inTable, arcpy.env.workspace, outTable, expression)

<Result 'E:\\My Drive\\School\\College\\06 Grad Full\\Semester 2\\Geospatial Programming\\Assignment 6\\interpolation.gdb\\yearly'>

In [16]:
## check is spatial analyst extension is available and then import it
try:
    if arcpy.CheckExtension("Spatial") == 'Available':
        arcpy.CheckOutExtension("Spatial")
    else:
        raise LicenseError
except LicenseError:
    print("Spatial Analyst License is not available")

In [14]:
from arcpy.sa import *

In [9]:
## geocode using lat long
yearly_points = 'yearly_points'
# arcpy.management.XYTableToPoint(outTable, outFeatureClass, 'Longitude', 'Latitude')

In [10]:
## create IDW for each year
for year in range(2009, 2019):
    attribute = "F" + str(year) + "_PREC"
    attribute_IDW = Idw(yearly_points, attribute)
    attribute_IDW.save(attribute)

In [10]:
descYearly = arcpy.Describe(yearly_points)
print(descYearly.extent.width, descYearly.extent.height)
width = descYearly.extent.width
height = descYearly.extent.height

cellSize = min(width, height) / 1000
print(cellSize)

23.383333332999996 12.866666666000015
0.012866666666000014


In [10]:
## create kriging for each year
field = "F2018_PREC"

# outKriging = Kriging(yearly_points, field, '#', cellSize)
# outKriging.save("F2018K")

In [11]:
## import Iowa counties
iowaCounties = folder + "/weather.gdb/iowa_counties"
# arcpy.FeatureClassToFeatureClass_conversion(iowaCounties, arcpy.env.workspace, "iowa_counties")

In [12]:
## describe iowa counties to get extent for clipping
descIowa = arcpy.Describe("iowa_counties")
rectangle = str(descIowa.extent.XMin) + " " + str(descIowa.extent.YMin) + " " + str(descIowa.extent.XMax) + " " + str(descIowa.extent.YMax)
rectangle

'-96.64070921375918 40.37194662123966 -90.14279675826373 43.501457447103576'

In [23]:
## clipping kriging into iowa counties
arcpy.Clip_management('F2018K', rectangle, 'F2018K_clipped', 'iowa_counties', '#', 
                      'ClippingGeometry', 'MAINTAIN_EXTENT')

<Result 'E:\\My Drive\\School\\College\\06 Grad Full\\Semester 2\\Geospatial Programming\\Assignment 6\\interpolation.gdb\\F2018K_clipped'>

In [15]:
# convert raster values to integers
outInt = Int("F2018K_clipped")
# outInt.save("F2018KC_Int")

In [16]:
min_F2018 = int(arcpy.management.GetRasterProperties(outInt, "MINIMUM").getOutput(0))
max_F2018 = int(arcpy.management.GetRasterProperties(outInt, "MAXIMUM").getOutput(0))

print(min_F2018)
print(max_F2018)

30
67


In [17]:
numofClasses = 5
eqInterval = (max_F2018 - min_F2018) / numofClasses
eqInterval

7.4

In [19]:
myremapRange = []
myBreak = min_F2018

for i in range(0, numofClasses):
    classCode = i + 1
    lowerBound = myBreak
    upperBound = myBreak + eqInterval
    remap = [lowerBound, upperBound, classCode]
    myremapRange.append(remap)
    myBreak += eqInterval

myremapRange

[[30, 37.4, 1], [37.4, 44.8, 2], [44.8, 52.199999999999996, 3], [52.199999999999996, 59.599999999999994, 4], [59.599999999999994, 67.0, 5]]

In [20]:
outReclassRR = Reclassify("F2018KC_Int", "Value", RemapRange(myremapRange), "NODATA")
outReclassRR.save("F2018_CL")

In [21]:
arcpy.RasterToPolygon_conversion("F2018_CL", "F2018_ismc", "NO_SIMPLIFY", "VALUE")

<Result 'E:\\My Drive\\School\\College\\06 Grad Full\\Semester 2\\Geospatial Programming\\Assignment 6\\interpolation.gdb\\F2018_ismc'>

In [ ]:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
